## Gear Optimization Notebook for Epic Seven
#### xx

##### Initialize Notebook Settings

In [ ]:
import setup as st
import fx_lib as fx
from datetime import datetime
import pandas as pd
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 40)
import numpy as np
import itertools
import pickle
import json
import yaml
with open(r'../inp/character_inputs.yaml') as file:
    target_stats = yaml.load(file, Loader=yaml.FullLoader)
with open('../inp/master_data.json') as json_file:
    data = json.load(json_file)
import set_combo as sc
from ipywidgets import interact, widgets, interactive, VBox, HBox
from IPython.display import display

In [ ]:
def set_widget4(Force_4Set, Speed, Attack, Counter, Lifesteal, Destruction, Rage, Revenge, Injury ):
    return
def set_widget2(Hit, Critical, Defense, Health, Resist, Immunity, Unity, Penetration ):
    return
def weight_widget(spd,atk,crit,cdmg,hp,defense,eff,res,ehp,dmg):
    return

In [ ]:
def run_widgets(hero_target,Force_4Set, char_list, st_setting):
    widgets.IntSlider(min=-1,max=3,value=1)
    w = interactive(weight_widget, 
                    spd = hero_target['SPD']['Weight'], atk = hero_target['ATK']['Weight'], 
                    crit = hero_target['CRIT']['Weight'], cdmg = hero_target['CDMG']['Weight'], 
                    hp = hero_target['HP']['Weight'], defense = hero_target['DEF']['Weight'], 
                    eff = hero_target['EFF']['Weight'], res = hero_target['RES']['Weight'],
                    ehp = hero_target['EHP']['Weight'], dmg = hero_target['Dmg_Rating']['Weight'])
    s4 = interactive(set_widget4,  Force_4Set = Force_4Set==1, Speed = 'Speed' in hero_target['include_sets'], Attack = 'Attack' in hero_target['include_sets'], Counter = 'Counter' in hero_target['include_sets'],  Lifesteal = 'Lifesteal' in hero_target['include_sets'], 
                     Destruction = 'Destruction' in hero_target['include_sets'], Rage = 'Rage' in hero_target['include_sets'], Revenge = 'Revenge' in hero_target['include_sets'], Injury = 'Injury' in hero_target['include_sets'])
    s2 = interactive(set_widget2, Hit = 'Hit' in hero_target['include_sets'], Critical = 'Critical' in hero_target['include_sets'], Defense = 'Defense' in hero_target['include_sets'], Health = 'Health' in hero_target['include_sets'], 
                     Resist = 'Resist' in hero_target['include_sets'], Immunity = 'Immunity' in hero_target['include_sets'], Unity = 'Unity' in hero_target['include_sets'], Penetration = 'Penetration' in hero_target['include_sets'])
    w1 = widgets.Dropdown(options=char_list,value=char,description="Character")
    w2 = widgets.IntSlider(value= st_setting[0], min=0, max=15, step=1, description='GEAR_ENHANCE:',
        disabled=False, continuous_update=False, orientation='horizontal', readout=True, readout_format='d')
    w3 = widgets.IntSlider(value= st_setting[1], min=0, max=30, step=1, description='GEAR_LIMIT:',
        disabled=False, continuous_update=False, orientation='horizontal', readout=True, readout_format='d')
    w4 = widgets.Checkbox(value= st_setting[2] ==1 , description="Keep gear currently equipped:")
    return w,s4,s2,w1,w2,w3,w4

def update_settings():
    include_sets = []
    for i in fx.set_4.Set_Nm.values:
        if s4.kwargs[i]: include_sets.extend([str(i)])
    for i in fx.set_2.Set_Nm.values:   
        if s2.kwargs[i]: include_sets.extend([str(i)])
    hero_target['include_sets'] = include_sets
    weight = w0.kwargs
    for i in weight:
        j = i.upper()
        if j == 'DEFENSE': j = 'DEF'
        elif j == 'DMG': j = 'Dmg_Rating'
        hero_target[j]['Weight'] = weight[i]
    return hero_target

In [ ]:
def stat_range():
    spd_range = widgets.IntRangeSlider(
        value=[80, 350], min=80, max=350, step=1,
        description='Speed:', disabled=False, continuous_update=False, orientation='horizontal',readout=True, readout_format='d')
    atk_range = widgets.IntRangeSlider(
        value=[0, 9999], min=0, max=9999, step=1,
        description='Attack:', disabled=False, continuous_update=False, orientation='horizontal',readout=True, readout_format='d')
    crit_range = widgets.IntRangeSlider(
        value=[0, 101], min=0, max=101, step=1,
        description='Crit Chance:', disabled=False, continuous_update=False, orientation='horizontal',readout=True, readout_format='d')
    cdmg_range = widgets.IntRangeSlider(
        value=[150, 500], min=150, max=500, step=1,
        description='Crit Dmg:', disabled=False, continuous_update=False, orientation='horizontal',readout=True, readout_format='d')
    hp_range = widgets.IntRangeSlider(
        value=[0, 35000], min=3000, max=35000, step=1,
        description='HP:', disabled=False, continuous_update=False, orientation='horizontal',readout=True, readout_format='d')
    def_range = widgets.IntRangeSlider(
        value=[0, 3500], min=300, max=3500, step=1,
        description='Defense:', disabled=False, continuous_update=False, orientation='horizontal',readout=True, readout_format='d')
    eff_range = widgets.IntRangeSlider(
        value=[0, 350], min=0, max=350, step=1,
        description='Effectiveness:', disabled=False, continuous_update=False, orientation='horizontal',readout=True, readout_format='d')
    res_range = widgets.IntRangeSlider(
        value=[0, 350], min=0, max=350, step=1,
        description='Resistance:', disabled=False, continuous_update=False, orientation='horizontal',readout=True, readout_format='d')
    s1 = [spd_range,atk_range,crit_range,cdmg_range,hp_range,def_range,eff_range,res_range]
    return [spd_range,atk_range,crit_range,cdmg_range,hp_range,def_range,eff_range,res_range]

In [ ]:
def print_gear_options(odf,s1,s2,w3):
    active_sets = []
    for i in range(1,len(s2)):
        if s2[i].value: active_sets.append(s2[i].description)
    op_cols = ['Complete','Set_1','Set_2','Set_3','WW','ATK','HP','DEF','SPD','CRIT','CDMG','EFF','RES', 'Dmg_Rating','EHP','CP']
    print_df = odf[op_cols][ (odf.SPD >= s1[1].value[0])&(odf.SPD <= s1[1].value[1]) &
                 (odf.ATK >= s1[2].value[0])&(odf.ATK <= s1[2].value[1]) &
                 (odf.CRIT >= s1[3].value[0])&(odf.CRIT <= s1[3].value[1]) &
                 (odf.CDMG >= s1[4].value[0])&(odf.CDMG <= s1[4].value[1]) &
                 (odf.HP >= s1[5].value[0])&(odf.HP <= s1[5].value[1]) &
                 (odf.DEF >= s1[6].value[0])&(odf.DEF <= s1[6].value[1]) &
                 (odf.EFF >= s1[7].value[0])&(odf.EFF <= s1[7].value[1]) &
                 (odf.RES >= s1[8].value[0])&(odf.RES <= s1[8].value[1]) &
                 (odf.Set_1.isin(active_sets)) & (odf.Set_2.isin(active_sets)) &
                 (odf.Set_3.isin(active_sets) | odf.Set_3.isna() ) &
                 ~(odf.Complete.isin(['PREVIOUS','CURRENT']))
                ].sort_values([w3.value],ascending=False).head(20).copy()
    return print_df

In [ ]:
def run_stat_selector(uniqueSets):
    w1 = stat_range()
    s1 = [widgets.Label("Stat Range")]
    for i in range(0,len(w1)):
        s1.append(w1[i])
    s2 = [widgets.Label("Sets:")]
    for x in range(len(uniqueSets)):
        var = widgets.Checkbox( value=True, description= str(uniqueSets[x]), disabled=False)
        s2.append(var)
    sort_cols = ['WW','ATK','HP','DEF','SPD','CRIT','CDMG','EFF','RES','Dmg_Rating','EHP','CP']
    w3 = widgets.RadioButtons(options= sort_cols, value='WW', description='Sort:', disabled=False)
    return s1,s2,w3

In [ ]:
def save_hero(df, gear_selected):
    reco_list = gear_selected['gear_list']
    df['reco'][(df.reco == char)] = ''
    df['reco'] = np.where( df.id.isin(reco_list) , char, df['reco'])
    df['hero'] = np.where((df.hero == char)&(df.hero != df.reco),'',df.hero)
    df = df.sort_values(by = ['hero','reco','Type','efficiency','enhance'])
    df.to_pickle('../outp/upd_items.pkl')
    return df
def save_final_data(df, reco_df):
    df[~(df.reco=='')][['start_loc','hero','efficiency','rating','reco','Type','slot','set','level','rarity','enhance','mainStat','subStat1','subStat2','subStat3','subStat4']].to_csv('../reco/gear_reco.csv')

    df['hero'] = np.where( df.reco!='', df['reco'], df['hero'])
    df.to_pickle('../outp/equip_potential.pkl')

    reco_df.to_csv('../reco/hero_stats.csv')
    df = df.sort_values(by = ['hero','Type','efficiency','enhance'])
    export2 = df[['efficiency','hero','enhance','slot','level','set','rarity','mainStat','subStat1','subStat2','subStat3','subStat4','id','p_id','locked']].to_dict('records')
    with open('../outp/upd_items.json', 'w') as fp: json.dump(export2, fp)
    return

#### These are required user input settings that can be edited here in the notebook.
Description of Settings:

| Syntax | Description |
| :----------- | :----------- |
| <b>Manual_Selection:</b> |  Primarily used for run in terminal, if Selection = 1, the user is prompted with a refined gear list to choose from for each hero | 
| <b>Gear_Limit:</b> |  To improve run time, the top 'n' gears for EACH slot is used in optimization | 
| <b>Auto_Adj_Gear_Limit:</b> |  Adjusts the number of gear pieces selected based on combinations to optimize run time vs available selections |
| <b>Use_Broken_Sets:</b> |  Only complete sets are used in gear optimization {default: 0} |
| <b>No_Equipped_Gear:</b> |  Set to 1 {default}: does not use any equipped gear.  To include gear equipped on other heroes set to 0. |
| <b>Keep_Current_Gear:</b> |  Keeps any currently equipped gear on the hero |
| <b>Gear_Enhance:</b> |  Enhance gear to minimum level for stat selection/optimization |
| <b>Flat_Sub:</b> |  Weight flat stat values in substats {range 0.0-1.0, default:0.8} |
| <b>Flat_Main:</b> |  Weight value attributed to {Necklace, Ring, Boots} with flat main stats {range 0.0-1.0, default:0.5} |
| <b>Min_Hero_Lvl:</b> |  Default Hero level of either 50 or 60 for stat calculation when the hero is not included in input file |

In [ ]:
st.MANUAL_SELECTION = 1   ##recommend keeping 1 while using this notebook
st.GEAR_LIMIT = 15 
st.AUTO_ADJ_GEAR_LIMIT = 1
## GEAR SELECTION
st.USE_BROKEN_SETS = 1  
st.NO_EQUIPPED_GEAR = 1  
st.KEEP_CURR_GEAR = 1
## STAT SELECTION
st.GEAR_ENHANCE = 12
st.FLAT_SUB = 0.8
st.FLAT_MAIN = 0.5
## HERO CRITERIA
st.MIN_LEVEL = 50 

In [ ]:
fx.verify_setup()

## Section 1

In [ ]:
## Here are the latest settings from character_inputs.yaml:
hero_order = target_stats['Hero_Order']
lock_gear = target_stats['Lock_Gear']
print("Heroes for optimization:", hero_order)
if st.NO_EQUIPPED_GEAR == 1:
    print("No equipped gear will be included in optimization.")
else:
    print("Gear on the following heroes is locked and cannot be stolen from another hero.", lock_gear)
print("Gear will be unlocked if new gear is equipped on that hero")
j=0

If you want to adjust the heroes above, uncomment the below cell and manually update heroes for optimization and locked gear

In [ ]:
# hero_order = ['Faithless Lidica','Dizzy','Falconer Kluri','Cermia']
# lock_gear = ['Lilias','Seaside Bellona']

In [ ]:
df_hero, char_list = fx.hero_json_to_df(hero_order, data)
sc.startup_message(lock_gear)

## Section 2

### SELECT HERO
The following section can be looped through for one hero at a time.  You can select the hero and customize some settings to deliver optimal gear selection.  Once recommended gear has been assigned to the hero, that gear will become locked and previously equipped gear will be released.  Data is saved at the end of each hero optimization and can be recovered in case of error when doing multiple heroes at once.

To recover data, uncomment and run the cell below:

In [ ]:
# sc.df_items = pd.read_pickle('../outp/upd_items.pkl')

Specify the hero and review settings

In [ ]:
Force_4Set, char, hero_target = sc.start_hero(hero_order[j], target_stats)
w0,s4,s2,w1,w2,w3,w4 = run_widgets(hero_target,Force_4Set, char_list, [st.GEAR_ENHANCE,st.GEAR_LIMIT,st.KEEP_CURR_GEAR])

Optional settings to change:

In [ ]:
VBox([w1,w4,w2])

Save if settings were changed:

In [ ]:
Force_4Set, char, hero_target = sc.start_hero(w1.value, target_stats)
st.GEAR_ENHANCE = w2.value
st.KEEP_CURR_GEAR = 1 if w4.value == True else 0

Select weighting of hero stats

In [ ]:
HBox([VBox( [widgets.Label("Stat Weights") , w0] ), VBox( [widgets.Label("4 Piece Sets") , s4] ), VBox( [widgets.Label("2 Piece Sets") , s2] )])

In [ ]:
hero_target = update_settings()

### Run Gear Combinations

In [ ]:
gear_comb_dict = fx.set_combo(sc.df_items[sc.df_items.set.isin(hero_target['include_sets'])], char, sc.l4, sc.l2)  ## Output gear_comb_dict[ [set_nm] , [type] , [ID] ]
sc_output = sc.set_combination_iterate(gear_comb_dict, fx.set_4[fx.set_4.Set_Nm.isin(hero_target['include_sets'])].Set_Nm.values , fx.set_2[fx.set_2.Set_Nm.isin(hero_target['include_sets'])].Set_Nm.values, Force_4Set)
print('Progress: Step 1/4 Complete.  Number of combinations found', len(sc_output))
print('For processing effiency, I would aim to keep combinations less than 1 million')

In [ ]:
sc_df = pd.DataFrame(sc_output, columns =['Set_1', 'Set_2', 'Set_3','Complete','Gear'])
sc_df['gear_list'] = sc_df.apply(lambda row: fx.gear_split(row) , axis=1)
sc_df[['0','1','2','3','4','5']] = pd.DataFrame(sc_df.gear_list.values.tolist(), index= sc_df.index)
sc_df = sc_df.drop_duplicates(['0','1','2','3','4','5'])

This step adds the currently equipped gear to the output for comparison.  Some of the hero's gear may have already been assigned to another hero during this optimization and cannot be selected for the desired output.

In [ ]:
### add current gear setting stats
current_gear = pd.DataFrame(columns = ['Set_1', 'Set_2', 'Set_3', 'Complete', 'Gear', 'gear_list', '0', '1', '2', '3', '4', '5' ], index = ['0'])
try:
    nix=0
    for gid in range(0,6):
        current_gear[str(gid)] = sc.df_items[(sc.df_items.hero == char) & (sc.df_items.Type == gid)][['id']].values
        val = sc.df_items[(sc.df_items.hero == char) & (sc.df_items.Type == gid)][['reco']].values
        if (val > '') & (val!=char): nix = 1
    current_gear = fx.get_set_bonus(current_gear, sc.df_items)
    if nix==1: current_gear['Complete'] = 'PREVIOUS'
    elif nix==0: current_gear['Complete'] = 'CURRENT'
    sc_df = sc_df.append(current_gear)
    sc_df = sc_df.reset_index()
    hero_with_gear = 1
except:
    hero_with_gear = 0

Calculates the stats for each combination of gear

In [ ]:
### final stats for each combination
output = fx.get_combo_stats(sc_df, df_hero, fx.mainst_sum(sc_df, sc.df_items), fx.subst_sum(sc_df, sc.df_items), \
                            fx.set_sum(sc_df), fx.bonus_eqp_sum(df_hero[df_hero.Name == char]), char, hero_target)
print('Progress: Step 2/4 Complete.  Number of unique combinations for optimization', len(output[output.Complete!='PREVIOUS']) )

In [ ]:
if hero_with_gear == 1:
    choice_df = output.iloc[-1:,:].copy()
    output = output.sort_values(by = ['WW'], ascending = False )  ## faster than inplace
else:
    output = output.sort_values(by = ['WW'], ascending = False )  ## faster than inplace
    choice_df = output.iloc[:1,:].copy()

## regular process
output2 = output.copy()
for stat in fx.set_df[fx.set_df.Bonus_Stat != 'NA'].Bonus_Stat.values:
    output2.drop( output2[(output2[stat]).astype(int) < hero_target[stat]['Min']].index, inplace=True)
    output2.drop( output2[(output2[stat]).astype(int) > hero_target[stat]['Max']].index, inplace=True)
print("Progress: Step 3/4 Complete.  The number of combinations available with stats in specified range is: ",len(output2))
if len(output2)==0:
    output2 = output.copy()
    print("Since no combinations meet criteria, best alternative combinations will be shown based on desired stat weighting.")

In [ ]:
## record top options for key stats
if st.MANUAL_SELECTION == 1:
    choice_df = choice_df.append(output.iloc[:3,:])
    choice_df = choice_df.append(output2.sort_values(by = ['WW'], ascending = False ).iloc[:3,:] )
    try: 
        for STAT in np.unique(hero_target['Prio']):
            choice_df = choice_df.append(output.sort_values(by = STAT, ascending = False ).iloc[:3,:] )
            choice_df = choice_df.append(output2.sort_values(by = STAT, ascending = False ).iloc[:3,:] )
    except: pass
try:
    for i in range(0,len(hero_target['Prio'])):
        target = hero_target['Prio'][i]
        cut_off = output2[target].quantile(0.9)
        output2 = output2[ output2[target] >= cut_off ]
except: pass
print("combinations after prio sortings ", len(output2) )

if st.MANUAL_SELECTION == 1:
    choice_df = choice_df.append(output2.sort_values(by = ['WW'], ascending = False ).iloc[:3,:] )
    choice_df = choice_df.append(output2[(output2.Set_1.isin(['Unity','Immunity','Penetration'])) | (output2.Set_2.isin(['Unity','Immunity'])) | \
            (output2.Set_3.isin(['Unity','Immunity']))].sort_values(by = ['WW'], ascending = False ).iloc[:3,:] )
    choice_df = choice_df.append(output2[(output2.Set_1.isin(['Counter','Lifesteal','Rage','Injury','Revenge']))].sort_values(by = ['WW'], ascending = False ).iloc[:3,:] )
    try: 
        for STAT in np.unique(hero_target['Prio']):
            choice_df = choice_df.append(output2.sort_values(by = STAT, ascending = False ).iloc[:3,:] )
    except: pass
    choice_df = choice_df.drop(axis = 1, columns = ['gear_list','Gear']).drop_duplicates()
idx_reco = output2.iloc[[0]].index.values[0]
odf = output.copy()
idx_text = 'Enter the index value for the gear combination with the hero stats you would like to apply:'
idx = widgets.BoundedIntText(value=idx_reco, min=0, max=len(odf)-2, description="Index:", disabled=False)
uniqueSets = (odf['Set_1'].dropna().append(odf['Set_2'].dropna()).append(odf['Set_3'].dropna())).unique()
s1,s2,w3 = run_stat_selector(uniqueSets)
box1 = VBox(s1)
box2 = VBox(s2)
box3 = VBox([widgets.Label("Choose the stat/column to sort by:"), w3] )

#### REVIEW GEAR OPTIONS

This table will display some of the top options based on the hero settings:

In [ ]:
choice_df[['Complete','Set_1','Set_2','Set_3','WW','ATK','HP','DEF','SPD','CRIT','CDMG','EFF','RES','Dmg_Rating','EHP']]

You can also interact with the stats to display the top options:

In [ ]:
HBox([box1,box2,box3])

#### SELECT FINAL GEAR SET

In [ ]:
print_df = print_gear_options(odf,s1,s2,w3)
print_df

In [ ]:
print(idx_text)
print("Value 0 to",len(odf)-2)
display(idx)

In [ ]:
print("Confirming index number:",idx.value)

Save the results for this hero:

In [ ]:
sc.df_items = save_hero(sc.df_items, output.loc[idx.value])
try: reco_df.append(output.loc[idx.value]['gear_list'])
except: reco_df = pd.DataFrame(output.loc[idx.value]['gear_list'])
j+=1

### Go Back to Run Next Hero

Save the final results:

In [ ]:
save_final_data(sc.df_items, reco_df)

### End of optimization notebook

In [ ]:
#gear_selected = output2.iloc[0,:]
row_idx = idx.value
gear_selected = output.loc[row_idx]
reco_list = gear_selected['gear_list']
try: reco_df.append(gear_selected)
except: reco_df = pd.DataFrame(gear_selected)
sc.df_items['reco'][(sc.df_items.reco == char)] = ''

sc.df_items['reco'] = np.where( sc.df_items.id.isin(reco_list) , char, sc.df_items['reco'])
sc.df_items['hero'] = np.where((sc.df_items.hero == char)&(sc.df_items.hero != sc.df_items.reco),'',sc.df_items.hero)

sc.df_items = sc.df_items.sort_values(by = ['hero','reco','Type','efficiency','enhance'])
sc.df_items.to_pickle('../outp/upd_items.pkl')

In [ ]:
sc.df_items[~(sc.df_items.reco=='')][['start_loc','hero','efficiency','rating','reco','Type','slot','set','level','rarity','enhance','mainStat','subStat1','subStat2','subStat3','subStat4']].to_csv('../reco/gear_reco.csv')

sc.df_items['hero'] = np.where( sc.df_items.reco!='', sc.df_items['reco'],sc.df_items['hero'])
sc.df_items.to_pickle('../outp/equip_potential.pkl')

reco_df.to_csv('../reco/hero_stats.csv')
sc.df_items = sc.df_items.sort_values(by = ['hero','Type','efficiency','enhance'])
export2 = sc.df_items[['efficiency','hero','enhance','slot','level','set','rarity','mainStat','subStat1','subStat2','subStat3','subStat4','id','p_id','locked']].to_dict('records')
with open('../outp/upd_items.json', 'w') as fp: json.dump(export2, fp)

recoup from error

In [ ]:
import pandas as pd
df_items = pd.read_pickle('../outp/upd_items.pkl')

In [ ]:
df_items[df_items.id.isin(odf.loc[row_idx].gear_list)].sort_values(by=['Type'])

In [ ]:
df_items[df_items.reco!=''][['start_loc','hero','efficiency','rating','reco','Type','slot','set','level','rarity','enhance','mainStat','subStat1','subStat2','subStat3','subStat4']].to_csv('../reco/gear_reco.csv')

df_items['hero'] = np.where( df_items.reco!='', df_items['reco'],df_items['hero'])
df_items.to_pickle('../outp/equip_potential.pkl')

df_items = df_items.sort_values(by = ['hero','Type','efficiency','enhance'])
export2 = df_items[['efficiency','hero','enhance','slot','level','set','rarity','mainStat','subStat1','subStat2','subStat3','subStat4','id','p_id','locked']].to_dict('records')
with open('../outp/upd_items.json', 'w') as fp: json.dump(export2, fp)

### Gear analysis

In [ ]:
df_lvl85 = sc.df_items[(sc.df_items.level == 85)].copy()

In [ ]:
## look at level 85 gear
df_lvl85.groupby(['level','enhance'])[['efficiency','maxp']].agg(['count','max','mean','min']) 

In [ ]:
# df_lvl85[(df_lvl85.slot == 'Boots')&(df_lvl85.main_tp == 'Spd')].sort_values(by=['efficiency'], ascending = False)
df_lvl85[(df_lvl85.enhance < 15)&(df_lvl85.SPD > 0)].sort_values(by=['maxp'], ascending = False)

#### Gear var

In [ ]:
# gear_chk = sc.df_items[(sc.df_items.main_tp == 'Spd')&(sc.df_items.enhance<15)].copy()
gear_chk = sc.df_items[(sc.df_items.level >= 75)&(sc.df_items.Type.isin([3,4]))].copy()

In [ ]:
gear_chk.groupby(['slot','main_tp','set'])[['efficiency','maxp']].agg(['count','max','mean','min']) 

#### Gear list

In [ ]:
# gear_chk = sc.df_items[(sc.df_items.main_tp == 'Spd')&(sc.df_items.enhance<15)].copy()
gear_chk = sc.df_items[(sc.df_items.level >= 75)&(sc.df_items.enhance<15)].copy()

In [ ]:
gear_chk.sort_values(['SPD'],ascending=False)

### Gear removal

##### choose gear to delete

In [ ]:
df_items = pd.read_pickle('../outp/equip_potential.pkl')

In [ ]:
x1 = df_items['efficiency'].quantile(q=0.4)
x2 = df_items['max_eff'].quantile(q=0.3)
x3 = df_items['rating'].quantile(q=0.4)
x4 = df_items['current_eff'].quantile(q=0.3)

In [ ]:
low_items = df_items[(df_items.efficiency <= x1) & (df_items.max_eff <= x2) & (df_items.rating <= x3) & (df_items.current_eff <= x4) & (df_items.SPD <= 8)].copy()


In [ ]:
print("number of items" , len(df_items))
print("number of unequipped items" , len(df_items[df_items.hero=='']))
print("number of items to delete" , len(low_items))
print("number of unequipped items to delete" , len(low_items[low_items.hero=='']))

In [ ]:
low_items

In [ ]:
low_items[low_items.hero==''][['id','p_id','efficiency','max_eff','set','slot','level','enhance']].to_csv('../outp/remove_list.csv')

##### remove gear from items

In [ ]:
remove = pd.read_csv('../outp/remove_list.csv')

In [ ]:
check1 = len(remove.id.values)
check2 = df_items[df_items.id.isin(remove.id.values)]['id'].count()

In [ ]:
if check1 == check2:
    print('pass', check1, check2)
    df_items = df_items.sort_values(by = ['hero','Type','efficiency','enhance'])
    export1 = df_items[~df_items.id.isin(remove.id.values)][['efficiency','hero','enhance','slot','level','set','rarity','mainStat','subStat1','subStat2','subStat3','subStat4','id','p_id','locked']].copy()
    export2 = export1.to_dict('records')
    with open('../outp/upd_items.json', 'w') as fp: json.dump(export2, fp)

In [ ]:
print( sc.df_items['id'].count() , export1['id'].count(), check1 )